# Generate the Dataset

### Connect to Imgur

In [ ]:
from imgurpython import ImgurClient

client_id = 'ea6214471d4bfd2'
client_secret = 'c53450f034a96c24f461054f9bebb8c0c470de1e'

client = ImgurClient(client_id, client_secret)

### [Run to see remaining credits]

In [ ]:
client.credits

### Scrape Comments

In [ ]:
user_list = ["LBarto", "JCentauri", 
            "TheBestThingSinceJimboSlicedBread", "TheGrinningOwl", 
            "CrystalShipsCometh", "ArtOzz", 
            "notacobra", "HatsfordaEmprah", 
            "SpecialAgentCharlesCarmichael", "SneakyEmu", 
            "AssistantToTheRegionalManagerDwightSchrute", "ThreeBodyProblemOnlyWithDicksInstead", 
            "Tootersmcgooters", "thankgodfordeaddinosaurs", 
            "14November", "OliverClothesoff70"]

Run the block below to scrape and store comments for each of the users in the list above.

In [ ]:
for user in user_list:
    
    # Cancel if running out of credits.
    if int(client.credits["UserRemaining"]) <= 5:
        print ("Cancelling scrape. Not enough credits remaining.")
        break
            
    # Get the number of comments for the user.
    num_comments = client.get_account_comment_count(user)
    
    # Calculate the number of pages.
    num_pages = num_comments//50
    
    if num_pages > 50: num_pages = 50
    
    # Loop through all the users's pages.
    for page in range(num_pages):
        
        if (page % 6) == 0:
            trainortest = "test"
        else:
            trainortest = "train"
        
        # Print out progress.
        print (f"Scraping comments for {user}. Page {page}/{num_pages}. Placing into the {trainortest} set.")
        
        # Cancel if running out of credits.
        if int(client.credits["UserRemaining"]) <= 5:
            print ("Cancelling scrape. Not enough credits remaining.")
            break
        
        # Get the comments for the current page.
        comments = client.get_account_comments(user, page=page)

        # Process the comments - Add <SOC to beginning, and <EOC> to the end of each comment.
        processed_comments = []
        for comment in comments:
            processed_comment = "SOC " + comment.comment + " EOC"
            processed_comments.append(processed_comment)

        # Write the page to a text file.
        with open(f"data/imgur-comments/{trainortest}/{user}-{page}.txt", "w") as text_file:
            print(" ".join(processed_comments), file=text_file)

# Train a Language Model

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.learner import *
from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

In [3]:
import random
import spacy
import untokenize
import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

In [4]:
PATH = "data/imgur-comments/"

TRN_PATH = "train/"
VAL_PATH = "test/"

TRN = f"{PATH}{TRN_PATH}"
VAL = f"{PATH}{VAL_PATH}"

### Create the Data Loader

In [5]:
TEXT = data.Field(lower=True, tokenize="spacy")

In [6]:
FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)

In [7]:
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=64, bptt=70, min_freq=10)

### Create the Model

In [8]:
em_sz = 200
nh = 500
nl = 3

In [9]:
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [10]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
               dropouti=0.2, dropout=0.2, wdrop=0.4, dropoute=0.08, dropouth=0.2)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [ ]:
learner.load_encoder('weights')

### Train the Model

In [11]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=1)
learner.save_encoder('weights')

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.967025   4.821035  



In [12]:
learner.fit(3e-3, 5, wds=1e-6, cycle_len=2)
learner.save_encoder('weights')

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.708063   4.527083  
    1      4.627142   4.458598                              
    2      4.582128   4.367992                              
    3      4.509277   4.324723                              
    4      4.499065   4.287951                              
    5      4.455843   4.257252                              
    6      4.465324   4.243248                              
    7      4.403016   4.21553                               
    8      4.436291   4.211142                              
    9      4.383798   4.184051                              



In [13]:
learner.fit(3e-3, 5, wds=1e-6, cycle_len=2)
learner.save_encoder('weights')

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.422339   4.190591  
    1      4.37027    4.163475                              
    2      4.393037   4.16837                               
    3      4.356156   4.145999                              
    4      4.386508   4.153205                              
    5      4.337894   4.131939                              
    6      4.368774   4.143419                              
    7      4.331461   4.120325                              
    8      4.366441   4.131748                              
    9      4.321885   4.110778                              



In [14]:
learner.fit(1e-3, 1, wds=1e-6, cycle_len=10)
learner.save_encoder('weights')

HBox(children=(IntProgress(value=0, description='Epoch', max=10), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.327815   4.112199  
    1      4.314948   4.106339                              
    2      4.326125   4.101321                              
    3      4.301637   4.093352                              
    4      4.278832   4.087898                              
    5      4.281264   4.083559                              
    6      4.278934   4.080408                              
    7      4.275993   4.078458                              
    8      4.267891   4.077152                              
    9      4.262315   4.076191                              



In [23]:
learner.fit(1e-3, 1, wds=1e-6, cycle_len=100)
learner.save_encoder('weights')

HBox(children=(IntProgress(value=0, description='Epoch'), HTML(value='')))

epoch      trn_loss   val_loss                              
    0      4.261642   4.076662  
    1      4.281069   4.076346                              
    2      4.260958   4.076161                              
    3      4.266041   4.076634                              
    4      4.253239   4.076164                              
    5      4.29051    4.085907                              
    6      4.2932     4.084022                              
    7      4.277769   4.079171                              
    8      4.278889   4.078976                              
    9      4.268984   4.074691                              
    10     4.266975   4.073494                              
    11     4.257127   4.069165                              
    12     4.262243   4.068845                              
    13     4.26747    4.066592                              
    14     4.252213   4.064995                              
    15     4.253332   4.06118                       

In [24]:
learner.save_encoder('weights')

# Test

In [25]:
import re

def process(combined_comments):
    
    # Split the comments into individual comments.
    comments = combined_comments.split("eoc")
    
    # Remove unneeded words.
    comments = [re.sub("<eos>", "", comment) for comment in comments]
    comments = [re.sub("soc", "", comment) for comment in comments]
    
    # Fix punctuation.
    comments = [re.sub(" !", "!", comment) for comment in comments]
    comments = [re.sub(" \.", ".", comment) for comment in comments]
    comments = [re.sub(" ,", ",", comment) for comment in comments]
    
    # Separate out comments with multiple sentences in them.
    all_comments = []    
    for comment in comments:        
        parts = comment.split(". ")
        parts = [part.capitalize() + "." for part in parts[:-1]] + [parts[-1].capitalize()]        
        for part in parts:
            all_comments.append(part)
    comments = all_comments
    
    # Remove whitespace.
    comments = [comment.lstrip() for comment in comments]
    comments = [re.sub(' +', ' ', comment) for comment in comments]  
    
    # Remove bad comments.
    comments = list(filter(None, comments))
    comments = list(filter(lambda comment: comment[0] not in ['!', '?'], comments))
    
    # Fix split words.
    comments = [re.sub(" s", "'s", comment) for comment in comments]
    comments = [re.sub(" 'm", "'m", comment) for comment in comments]
    comments = [re.sub(" 'l", "'l", comment) for comment in comments]
    comments = [re.sub(" 'd", "'d", comment) for comment in comments]
    comments = [re.sub(" 'r", "'r", comment) for comment in comments]
    comments = [re.sub(" ’r", "'r", comment) for comment in comments]
    comments = [re.sub(" 's", "'s", comment) for comment in comments]
    comments = [re.sub(" ’s", "'s", comment) for comment in comments]
    comments = [re.sub(" n'", "n'", comment) for comment in comments]
    comments = [re.sub(" 'v", "'v", comment) for comment in comments]
    
    # Fix broken words.
    comments = [re.sub("do nt", "dont", comment) for comment in comments] 
    
    # Fix random apostrophes.
    comments = [re.sub("u's", "u s", comment) for comment in comments] 
    comments = [re.sub("e's", "e s", comment) for comment in comments] 
    comments = [re.sub("I's", "I s", comment) for comment in comments]
    comments = [re.sub("o's", "o s", comment) for comment in comments]
    comments = [re.sub("m's", "m s", comment) for comment in comments]
    
    # Capitalize the first word.
    comments = [comment.capitalize() for comment in comments] 
    
    # Fix decapitalised.
    comments = [re.sub(" i ", " I ", comment) for comment in comments] 
    comments = [re.sub(" i'", " I'", comment) for comment in comments] 
    
    return comments

In [36]:
spacy_tok = spacy.load('en')
m=learner.model
ss="""soc"""
s = [spacy_tok(ss)]
t=TEXT.numericalize(s)

In [37]:
# Set batch size to 1
m[0].bs=1
# Turn off dropout
m.eval()
# Reset hidden state
m.reset()
# Get predictions from model
res,*_ = m(t)
# Put the batch size back to what it was
m[0].bs=64

In [58]:
for i in range(10000):
    
    # Generate candidates from 
    candidates=res[-1].topk(5)[1]   
    
    # Keep polling for a new word until no <unk>.
    n = 0
    while n is 0:
        next_word = random.choice(candidates)        
        n = next_word.data[0]
        
    # Add the new word to the sentence.
    ss = ss + " " + TEXT.vocab.itos[next_word.data[0]]
    
    # Poll the model again.
    res,*_ = m(next_word[0].unsqueeze(0))
    
processed_ss = process(ss)

In [59]:
def get_comment():
    
    for i in range(len(processed_ss)):    
        yield processed_ss[i]    

In [60]:
gc = get_comment()

In [68]:
for i in range(20):
    print (f"=> {next(gc)}")

=> I've heard it's a bit of the issue in this post and not even the original.
=> But that's a lot more of the time I read this.
=> Thanks op! :) I'll have to go get this 
=> .
=> It is.
=> I've been there, but it'seems like it doesn't have a good experience 
=> You aren't.
=> The guy isn't a fucking idiot.
=> He s a dick but you can be a little sad about that and he'll be the first 
=> Oh I do.
=> It would be the only thing you would know to do in this world and not be able to do this 
=> That is an interesting post! I've got a few things.
=> I'm sure they can be better.
=> I dont have an issue but that is the most annoying way 
=> I think it was just an insult, I'm a guy 
=> But I do.
=> You know what's wrong is this, it makes it look like they are trying and not the other guy who wasn't in it and it did not have the time of life 
=> .
=> I have a lot.
=> It doesn't'seem a lot to do with the fact I wasn't aware.
